# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [1]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="test-bed-fltk-group16-mb"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Scale the default-node-pool up.

In [2]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.


Updated [https://container.googleapis.com/v1/projects/test-bed-fltk-group16-mb/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [3]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1023 15:48:34.073902    9654 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found


: 1

## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [4]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1023 15:48:38.441297    9676 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" does not exist. Installing it now.
NAME: extractor
LAST DEPLOYED: Sun Oct 23 15:48:40 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [19]:
# HOMEWORKS
# EXPERIMENT_FILE="/home/mattiacs/Documents/Developer/Edu/TUDelft/Year_4/Quarter_1/CS4215-QPECS/homeworks_2/ex6_configs/distributed_4_node.json"

# PROJECT
# EXPERIMENT_FILE="../configs/federated_tasks/example_arrival_config.json"
# EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
# CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

EXPERIMENT_FILE="../project/configs/distributed/mnist_fashionmnistcnn/epoch_25/experiment_10e-3_25.json"
CLUSTER_CONFIG="../project/configs/project_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [20]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1023 17:56:41.759289   18561 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled
W1023 17:56:46.696105   18583 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Sun Oct 23 17:56:49 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [15]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1023 17:53:51.715232   18268 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-23-2022 14:51:58 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-23-2022 14:51:58 root         INFO     Starting in cluster mode.
10-23-2022 14:51:58 root         INFO     Starting with experiment replication: 0 with seed: 42
10-23-2022 14:51:58 root         INFO     Starting as Orchestrator
10-23-2022 14:51:58 root         INFO     Starting Orchestrator, initializing resources....
10-23-2022 14:51:58 root         INFO     Loading in cluster configuration file
10-23-2022 14:51:58 root         INFO     Pointing configuration to in cluster configuration.
10-23-2022 14:51:58 root         INFO     Starting cluster manager
10-23-2022 14:51:58 ClusterManager INFO     Spinning up cluster manag

10-23-2022 14:55:05 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:55:05 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:55:05 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:55:10 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:55:15 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:55:15 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:55:20 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:55:25 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:55:25 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:55:30 root      

10-23-2022 14:59:01 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:59:03 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:59:06 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:59:11 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:59:14 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:59:14 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:59:16 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:59:21 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 14:59:24 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 14:59:26 root      

10-23-2022 15:02:57 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:03:02 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:03:02 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:03:07 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:03:12 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:03:12 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:03:17 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:03:22 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:03:22 ResourceWatchDog INFO     Fetching pod information of 

10-23-2022 15:06:53 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:06:58 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:07:00 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:07:03 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:07:08 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:07:10 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:07:13 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:07:18 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:07:21 ResourceWatchDog INFO     Fetching pod information of 

...

10-23-2022 15:26:42 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:26:44 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:26:49 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:26:53 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:26:54 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:26:59 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:27:03 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:27:04 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:27:09 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 

10-23-2022 15:30:40 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:30:41 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:30:45 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:30:50 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:30:51 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:30:55 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:31:00 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:31:01 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:31:05 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 

10-23-2022 15:34:36 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:34:39 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:34:41 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:34:46 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:34:50 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:34:51 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:34:56 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:35:00 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:35:01 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 

10-23-2022 15:38:38 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:38:38 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:38:38 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:38:43 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:38:48 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:38:48 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:38:53 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:38:58 root         INFO     Waiting for 9a7ddfd7-6eee-4694-acfb-f923de6bee62 to complete, 0 pending, 0 arrivals
10-23-2022 15:38:59 ResourceWatchDog INFO     Fetching pod information of cluster...
10-23-2022 15:39:03 root      

10-23-2022 15:42:24 ClusterManager INFO     Successfully stopped execution of ClusterManager
10-23-2022 15:42:24 root         INFO     Stopped execution of Orchestrator replication: 0.


In [16]:
CURRENT_POD_ID="9a7ddfd7-6eee-4694-acfb-f923de6bee62"
# To get logs from learners (example)
kubectl logs -n test trainjob-$CURRENT_POD_ID-master-0

W1023 17:54:27.394461   18330 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-23-2022 14:52:07 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 14:52:07 root         INFO     Starting in client mode
10-23-2022 14:52:07 root         INFO     Starting with host=trainjob-9a7ddfd7-6eee-4694-acfb-f923de6bee62-master-0 and port=23456
10-23-2022 14:52:07 root         INFO     Initializing backend for training process: gloo
10-23-2022 14:52:10 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-23-2022 14:52:10 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-23-2022 14:52:10 root         INFO     Starting Creating client with 0
10-23-2022 14:52:10 Client-0-9

10-23-2022 15:39:04 Client-0-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [24,    50] loss: 0.182
10-23-2022 15:39:52 Client-0-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [24,   100] loss: 0.173
10-23-2022 15:40:16 Client-0-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [25,     0] loss: 0.005
10-23-2022 15:41:05 Client-0-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [25,    50] loss: 0.177
10-23-2022 15:41:53 Client-0-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [25,   100] loss: 0.168
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=110737, duration_test=7906, loss_train=0.6289124763011933, accuracy=81.08, loss=11.011719942092896, class_precision=array([0.78714859, 0.98449612, 0.67986799, 0.74683544, 0.73728814,
       0.8458498 , 0.52941176, 0.91304348, 0.95555556, 0.91769547]), class_recall=array([0.72592593, 0.94776119, 0.76579926, 0.87407407, 0.68235294,
       0.93043478, 0.4479638 , 0.86065574, 0.91101695, 0.94092827]), confusion_mat=array([

       0.95652174, 0.71945701, 0.93442623, 0.97457627, 0.95780591]), confusion_mat=array([[192,   0,   3,  11,   2,   2,  56,   0,   4,   0],
       [  0, 260,   1,   6,   0,   0,   0,   0,   1,   0],
       [  1,   1, 201,   0,  17,   1,  47,   0,   1,   0],
       [  5,   2,   3, 243,   5,   0,  12,   0,   0,   0],
       [  1,   1,  20,  12, 188,   0,  33,   0,   0,   0],
       [  0,   0,   0,   2,   0, 220,   0,   6,   0,   2],
       [ 21,   0,  14,   9,  15,   0, 159,   0,   3,   0],
       [  0,   0,   0,   0,   0,   7,   0, 228,   0,   9],
       [  0,   1,   0,   1,   0,   2,   2,   0, 230,   0],
       [  0,   0,   0,   0,   0,   2,   0,   8,   0, 227]])), EpochData(epoch_id=9, num_epochs=26, duration_train=1071229, duration_test=7708, loss_train=0.2907572665810585, accuracy=86.36, loss=7.5632151663303375, class_precision=array([0.87214612, 0.98490566, 0.83817427, 0.86572438, 0.83982684,
       0.94396552, 0.52272727, 0.9382716 , 0.9623431 , 0.9539749 ]), class_recall=array(

       0.96491228, 0.55479452, 0.95041322, 0.98297872, 0.95867769]), class_recall=array([0.74074074, 0.97761194, 0.79925651, 0.90740741, 0.79215686,
       0.95652174, 0.73303167, 0.94262295, 0.97881356, 0.97890295]), confusion_mat=array([[200,   0,   3,   7,   2,   0,  55,   0,   3,   0],
       [  0, 262,   1,   3,   0,   0,   1,   0,   1,   0],
       [  3,   1, 215,   0,  15,   1,  34,   0,   0,   0],
       [  5,   0,   3, 245,   4,   0,  13,   0,   0,   0],
       [  1,   1,  18,   8, 202,   0,  25,   0,   0,   0],
       [  0,   0,   0,   1,   0, 220,   0,   7,   0,   2],
       [ 20,   0,  13,   7,  19,   0, 162,   0,   0,   0],
       [  0,   0,   0,   0,   0,   6,   0, 230,   0,   8],
       [  0,   1,   0,   1,   0,   1,   2,   0, 231,   0],
       [  0,   0,   0,   0,   0,   0,   0,   5,   0, 232]])), EpochData(epoch_id=17, num_epochs=26, duration_train=2039043, duration_test=7986, loss_train=0.21593155831098557, accuracy=87.96, loss=7.005424067378044, class_precision=array

       [  0,   0,   0,   0,   0,   1,   0,   6,   0, 230]])), EpochData(epoch_id=24, num_epochs=26, duration_train=2876533, duration_test=6818, loss_train=0.17329949587583543, accuracy=88.32, loss=6.941479504108429, class_precision=array([0.86864407, 0.99242424, 0.87966805, 0.9037037 , 0.83673469,
       0.96491228, 0.56      , 0.95081967, 0.99137931, 0.95833333]), class_recall=array([0.75925926, 0.97761194, 0.78810409, 0.9037037 , 0.80392157,
       0.95652174, 0.760181  , 0.95081967, 0.97457627, 0.97046414]), confusion_mat=array([[205,   0,   3,   6,   2,   0,  53,   0,   1,   0],
       [  0, 262,   1,   3,   0,   0,   1,   0,   1,   0],
       [  3,   1, 212,   0,  16,   1,  36,   0,   0,   0],
       [  6,   0,   1, 244,   5,   0,  14,   0,   0,   0],
       [  1,   0,  15,   8, 205,   0,  26,   0,   0,   0],
       [  0,   0,   0,   1,   0, 220,   0,   6,   0,   3],
       [ 21,   0,   9,   7,  16,   0, 168,   0,   0,   0],
       [  0,   0,   0,   0,   0,   5,   0, 232,   0,   7

In [17]:
# To get logs from learners (federated learning)
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-0

W1023 17:55:53.558006   18444 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-23-2022 14:52:10 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 14:52:10 root         INFO     Starting in client mode
10-23-2022 14:52:10 root         INFO     Starting with host=trainjob-9a7ddfd7-6eee-4694-acfb-f923de6bee62-master-0 and port=23456
10-23-2022 14:52:10 root         INFO     Initializing backend for training process: gloo
10-23-2022 14:52:10 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 1
10-23-2022 14:52:10 torch.distributed.distributed_c10d INFO     Rank 1: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-23-2022 14:52:10 root         INFO

10-23-2022 15:38:16 Client-1-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [24,     0] loss: 0.005
10-23-2022 15:39:04 Client-1-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [24,    50] loss: 0.182
10-23-2022 15:39:52 Client-1-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [24,   100] loss: 0.173
10-23-2022 15:40:16 Client-1-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [25,     0] loss: 0.005
10-23-2022 15:41:05 Client-1-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [25,    50] loss: 0.177
10-23-2022 15:41:53 Client-1-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [25,   100] loss: 0.168
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=110780, duration_test=7801, loss_train=0.6289124763011933, accuracy=81.84, loss=10.726437509059906, class_precision=array([0.78723404, 0.97265625, 0.69818182, 0.76369863, 0.73953488,
       0.85665529, 0.64179104, 0.84860558, 0.93534483, 0.912     ]), class_recall=array([0.77731092, 0.92565056, 0.80334728, 0.86770428, 0.6

       0.95255474, 0.59567901, 0.91472868, 0.9535865 , 0.96341463]), class_recall=array([0.73529412, 0.94423792, 0.80334728, 0.89883268, 0.72608696,
       0.9738806 , 0.75984252, 0.9254902 , 0.95762712, 0.93307087]), confusion_mat=array([[175,   1,   3,  11,   0,   0,  45,   0,   3,   0],
       [  1, 254,   2,  10,   0,   0,   2,   0,   0,   0],
       [  1,   0, 192,   2,  16,   0,  26,   0,   2,   0],
       [  0,   5,   2, 231,   1,   0,  18,   0,   0,   0],
       [  0,   0,  15,  12, 167,   0,  35,   0,   1,   0],
       [  0,   0,   0,   0,   0, 261,   0,   6,   0,   1],
       [ 22,   0,  15,  11,   9,   0, 193,   0,   4,   0],
       [  0,   0,   0,   0,   0,  11,   0, 236,   0,   8],
       [  0,   0,   1,   1,   1,   2,   4,   1, 226,   0],
       [  0,   0,   0,   0,   0,   0,   1,  15,   1, 237]])), EpochData(epoch_id=9, num_epochs=26, duration_train=1071272, duration_test=7913, loss_train=0.2907572665810585, accuracy=87.04, loss=6.977570593357086, class_precision=array([

       0.97761194, 0.63548387, 0.92015209, 0.96610169, 0.96356275]), class_recall=array([0.74789916, 0.94795539, 0.84518828, 0.90272374, 0.7826087 ,
       0.97761194, 0.77559055, 0.94901961, 0.96610169, 0.93700787]), confusion_mat=array([[178,   1,   3,   9,   0,   0,  45,   0,   2,   0],
       [  2, 255,   0,   9,   1,   0,   2,   0,   0,   0],
       [  1,   0, 202,   2,  12,   0,  21,   0,   1,   0],
       [  0,   4,   4, 232,   1,   0,  16,   0,   0,   0],
       [  0,   0,  11,  12, 180,   0,  26,   0,   1,   0],
       [  0,   0,   0,   0,   0, 262,   0,   5,   0,   1],
       [ 18,   0,  16,   8,  12,   0, 197,   0,   3,   0],
       [  0,   0,   0,   0,   0,   6,   0, 242,   0,   7],
       [  0,   0,   1,   1,   1,   0,   2,   2, 228,   1],
       [  0,   0,   0,   0,   0,   0,   1,  14,   1, 238]])), EpochData(epoch_id=17, num_epochs=26, duration_train=2039088, duration_test=7885, loss_train=0.21593155831098557, accuracy=88.48, loss=6.400479704141617, class_precision=array

       [  0,   0,   0,   0,   0,   1,   1,  15,   1, 236]])), EpochData(epoch_id=24, num_epochs=26, duration_train=2876578, duration_test=7900, loss_train=0.17329949587583543, accuracy=88.52, loss=6.288822650909424, class_precision=array([0.88669951, 0.98084291, 0.85714286, 0.84558824, 0.86829268,
       0.98120301, 0.63924051, 0.91729323, 0.97435897, 0.95934959]), class_recall=array([0.75630252, 0.95167286, 0.82845188, 0.89494163, 0.77391304,
       0.9738806 , 0.79527559, 0.95686275, 0.96610169, 0.92913386]), confusion_mat=array([[180,   1,   3,   9,   0,   0,  43,   0,   2,   0],
       [  2, 256,   0,   9,   1,   0,   1,   0,   0,   0],
       [  2,   0, 198,   2,  12,   0,  25,   0,   0,   0],
       [  3,   4,   4, 230,   2,   0,  14,   0,   0,   0],
       [  0,   0,  12,  12, 178,   0,  28,   0,   0,   0],
       [  0,   0,   0,   0,   0, 261,   0,   5,   0,   2],
       [ 16,   0,  13,   9,  11,   0, 202,   0,   3,   0],
       [  0,   0,   0,   0,   0,   4,   0, 244,   0,   7

In [18]:
kubectl logs -n test trainjob-$CURRENT_POD_ID-worker-1

W1023 17:56:07.500187   18468 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Defaulted container "pytorch" out of: pytorch, init-pytorch (init)
10-23-2022 14:52:09 root         INFO     Loading file config/configuration.fltk.json
10-23-2022 14:52:09 root         INFO     Starting in client mode
10-23-2022 14:52:09 root         INFO     Starting with host=trainjob-9a7ddfd7-6eee-4694-acfb-f923de6bee62-master-0 and port=23456
10-23-2022 14:52:09 root         INFO     Initializing backend for training process: gloo
10-23-2022 14:52:10 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 2
10-23-2022 14:52:10 torch.distributed.distributed_c10d INFO     Rank 2: Completed store-based barrier for key:store_based_barrier_key:1 with 4 nodes.
10-23-2022 14:52:10 root         INFO

10-23-2022 15:38:16 Client-2-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [24,     0] loss: 0.005
10-23-2022 15:39:04 Client-2-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [24,    50] loss: 0.182
10-23-2022 15:39:52 Client-2-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [24,   100] loss: 0.173
10-23-2022 15:40:16 Client-2-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [25,     0] loss: 0.005
10-23-2022 15:41:05 Client-2-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [25,    50] loss: 0.177
10-23-2022 15:41:53 Client-2-9a7ddfd7-6eee-4694-acfb-f923de6bee62 INFO     [25,   100] loss: 0.168
No argument path is provided.
[EpochData(epoch_id=1, num_epochs=26, duration_train=110782, duration_test=7793, loss_train=0.6289124763011933, accuracy=80.72, loss=11.386070013046265, class_precision=array([0.79824561, 0.99086758, 0.6584507 , 0.70607029, 0.71138211,
       0.872     , 0.59      , 0.89919355, 0.93822394, 0.92490119]), class_recall=array([0.74590164, 0.94347826, 0.7991453 , 0.87698413, 0.6

       0.93775934, 0.59159159, 0.91538462, 0.9509434 , 0.96296296]), class_recall=array([0.68852459, 0.97826087, 0.78205128, 0.90079365, 0.72030651,
       0.95762712, 0.73234201, 0.93700787, 0.9298893 , 0.93975904]), confusion_mat=array([[168,   0,   5,  17,   1,   2,  50,   0,   1,   0],
       [  0, 225,   0,   3,   1,   0,   0,   0,   1,   0],
       [  1,   0, 183,   6,  24,   0,  20,   0,   0,   0],
       [  0,   2,   5, 227,   5,   0,  13,   0,   0,   0],
       [  0,   1,  15,  14, 188,   0,  42,   0,   1,   0],
       [  0,   0,   0,   0,   0, 226,   0,   8,   0,   2],
       [ 21,   1,  13,  12,  15,   0, 197,   0,  10,   0],
       [  0,   0,   0,   0,   0,  10,   0, 238,   0,   6],
       [  1,   0,   0,   4,   0,   1,  10,   2, 252,   1],
       [  0,   0,   0,   0,   0,   2,   1,  12,   0, 234]])), EpochData(epoch_id=9, num_epochs=26, duration_train=1071275, duration_test=8710, loss_train=0.2907572665810585, accuracy=85.72, loss=7.561816424131393, class_precision=array([

       0.95319149, 0.61919505, 0.90977444, 0.96268657, 0.97107438]), class_recall=array([0.70901639, 0.97826087, 0.81196581, 0.90079365, 0.72796935,
       0.94915254, 0.74349442, 0.95275591, 0.95202952, 0.9437751 ]), confusion_mat=array([[173,   0,   4,  13,   1,   2,  50,   0,   1,   0],
       [  0, 225,   1,   3,   0,   0,   0,   0,   1,   0],
       [  1,   0, 190,   5,  20,   0,  18,   0,   0,   0],
       [  1,   2,   3, 227,   5,   0,  14,   0,   0,   0],
       [  0,   1,  20,  15, 190,   0,  34,   0,   1,   0],
       [  0,   0,   0,   0,   0, 224,   0,   9,   0,   3],
       [ 22,   1,  14,  11,  14,   0, 200,   0,   7,   0],
       [  0,   0,   0,   0,   0,   8,   0, 242,   0,   4],
       [  0,   0,   0,   4,   0,   0,   6,   3, 258,   0],
       [  0,   0,   0,   0,   0,   1,   1,  12,   0, 235]])), EpochData(epoch_id=17, num_epochs=26, duration_train=2039090, duration_test=8181, loss_train=0.21593155831098557, accuracy=86.72, loss=6.959850013256073, class_precision=array

       [  0,   0,   0,   0,   0,   1,   1,  11,   0, 236]])), EpochData(epoch_id=24, num_epochs=26, duration_train=2876578, duration_test=7803, loss_train=0.17329949587583543, accuracy=87.08, loss=6.883911088109016, class_precision=array([0.87864078, 0.98253275, 0.82666667, 0.83088235, 0.83261803,
       0.95744681, 0.6231003 , 0.92045455, 0.97338403, 0.96721311]), class_recall=array([0.74180328, 0.97826087, 0.79487179, 0.8968254 , 0.74329502,
       0.95338983, 0.76208178, 0.95669291, 0.94464945, 0.94779116]), confusion_mat=array([[181,   0,   2,   9,   1,   2,  48,   0,   1,   0],
       [  0, 225,   1,   3,   0,   0,   0,   0,   1,   0],
       [  2,   0, 186,   5,  19,   0,  22,   0,   0,   0],
       [  1,   2,   4, 226,   5,   0,  14,   0,   0,   0],
       [  0,   1,  17,  15, 194,   0,  34,   0,   0,   0],
       [  0,   0,   0,   0,   0, 225,   0,   8,   0,   3],
       [ 21,   1,  13,  10,  14,   0, 205,   0,   5,   0],
       [  0,   0,   0,   0,   0,   7,   0, 243,   0,   4

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [76]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")

kubectl cp -n test $EXTRACTOR_POD_NAME:/opt/federation-lab/logging ../logging

W1019 15:42:31.888203   14421 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1019 15:42:32.776003   14448 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
tar: Removing leading `/' from member names


# Cleanup

## Removing orchestrator

In [77]:
helm uninstall -n test experiment-orchestrator

W1019 15:42:53.020634   14486 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [78]:
helm uninstall extractor -n test

W1019 15:42:58.179859   14502 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [79]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1019 15:43:21.797054   14527 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-0203a077-858f-4427-ae7f-9c7ad5b92c71" deleted
pytorchjob.kubeflow.org "trainjob-041637b0-9be2-4a36-bc1f-0b8488d1fdd7" deleted
pytorchjob.kubeflow.org "trainjob-0a65e275-2e1f-421a-9f58-b6a809f81e41" deleted
pytorchjob.kubeflow.org "trainjob-1be7dd3d-d117-48aa-a5a2-190743e8d845" deleted
pytorchjob.kubeflow.org "trainjob-20908ee6-abae-4b99-b508-3f36dd9161ee" deleted
pytorchjob.kubeflow.org "trainjob-490cdeec-195f-45b0-94bf-fac30ce4e481" deleted
pytorchjob.kubeflow.org "trainjob-62e17e42-a74b-40b6-932d-568c69ea8c84" deleted
pytorchjob.kubeflow.org "trainjob-62ef1b19-cbb7-49d4-ae48-b74a1f16afa0" deleted
pytorchjob.kubeflow.org "trainjob-6cc6085b-4f8d-414f-b430-1e504f8a9b6f" deleted
pytorchjob.kubeflow.org "trai